In [2]:
import os
import numpy as np

from sunpy.net import Fido, attrs as a
import astropy.units as u

import sunpy.map
from astropy.io import fits

from aiapy.calibrate import register, update_pointing, correct_degradation
from aiapy.calibrate.util import get_pointing_table, get_correction_table
import aiapy.psf

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import cv2
from PIL import Image

import shutil

import time
import datetime
from dateutil.relativedelta import relativedelta

from concurrent.futures import ThreadPoolExecutor



### Downloading the Data

#### Set time range

In [3]:
start_date = '2012/01/01 00:00:00'
end_date = '2024/01/01 00:00:00'

# Set Time Range
time_range = a.Time(start_date, end_date)

#### Download Directory

In [4]:
# Specify the directory where you want to download the files
download_directory = 'C:\Sun Data'
os.makedirs(download_directory, exist_ok=True)

#### Search for fits on VSO

In [ ]:


all_results = []

start_date_dt = datetime.datetime.strptime(start_date, '%Y/%m/%d %H:%M:%S')
end_date_dt = datetime.datetime.strptime(end_date, '%Y/%m/%d %H:%M:%S')

current_date = start_date_dt

while current_date < end_date_dt:
    next_date = current_date + relativedelta(months=1)
    if next_date > end_date_dt:
        next_date = end_date_dt
    try:
        time_range = a.Time(current_date.strftime('%Y/%m/%d %H:%M:%S'), next_date.strftime('%Y/%m/%d %H:%M:%S'))
        print("Searching for: ", time_range)
        results = Fido.search(
            time_range,
            a.Instrument.aia,
            a.Wavelength(193 * u.angstrom),
            a.Sample(60 * u.minute)
        )
        print("Found", results.file_num, "results")
        all_results.extend(results)
        current_date = next_date
    except Exception as e:
        print("Trying again!")
        continue


print(len(all_results))


#### Download files with error checking

In [ ]:
for results in all_results:
    downloaded_files = []
    # First Download attempt
    try:
        downloaded_files = Fido.fetch(
            results, path=download_directory)
    except Exception as e:
        print(
            f"Failed to download files. Retrying...")
    # Retrying the files that didnt download 10 times
    for i in range (20):
        time.sleep(5)
        if len(downloaded_files) == len(results):
            break
        try:
            downloaded_files = Fido.fetch(
                downloaded_files, path=download_directory)
        except Exception as e:
            print(
                f"Failed to download files. Retrying...")
        

# Output the list of downloaded files
print(f"Downloaded {len(downloaded_files)} files.")

### Quality checking and Re-downloading

#### First we check quality of all files, if the quality is less than 0 we delete the file and save the date and time in an array.
#### Then we redownload files 5 minutes after those files 
#### Then we re-check the quality of those newly downloaded files
#### Then we delete all the bad quality files

In [ ]:

# get list of all files, start date and end date and returns list of missing files
def get_missing_files(aia_files, start_date, end_date):
    # Making list of all dates of the files
    file_dates = convert_file_list_to_datetime(aia_files)
    
    
    # Making list of all expected dates
    start_date_dt = datetime.datetime.strptime(start_date, '%Y/%m/%d %H:%M:%S')
    end_date_dt = datetime.datetime.strptime(end_date, '%Y/%m/%d %H:%M:%S')

    expected_dates = []
    # making list of all dates from start to end with a gap of 1 hour
    for i in range(int((end_date_dt - start_date_dt).days * 24)):
        start_date_dt = start_date_dt + relativedelta(hours=1)
        expected_dates.append(start_date_dt)

    # get the missing dates
    missing_dates = []
    for date in expected_dates:
        if date not in file_dates:
            missing_dates.append(date)
    
    return missing_dates
    
    
# Function to check the quality of the files and delete the bad quality files, returns list of bad quality files
def check_file_quality(file_list, download_directory):
    bad_quality_files = []
    # open all files and check the quality
    for file in file_list:
        try:
            with fits.open(file) as hdul:
                header = hdul[1].header
                # Check if the quality is bad add to the list
                if header['QUALITY'] != 0:
                    bad_quality_files.append(file)
                    print(f"Bad quality file: {file}")
                hdul.close()
        except Exception as e:
            print(f"Failed to open {file}: {e}")
            continue
    # delete all bad quality files
    for bad_file in bad_quality_files:
        os.remove(bad_file)
        
    # Save the list of bad quality files to a text file
    bad_quality_files_file = os.path.join(download_directory, 'bad_quality_files.txt')
    with open(bad_quality_files_file, 'w') as f:
        for file in bad_quality_files:
            f.write(f"{file}\n")
            
    # return the list of bad quality files
    return bad_quality_files

# Function to get path of AIA files and converts it to list of datetime
def convert_file_list_to_datetime(file_list):
    file_dates = []
    for i in range(len(file_list)):
        # To remove path
        bad_file_list = file_list[i].split('\\')
        # Get just the file name
        bad_file = bad_file_list[-1]
        # Get the date and time of the bad quality file
        year = bad_file[14:18]
        month = bad_file[19:21]
        day = bad_file[22:24]
        hour = bad_file[25:27]
        date = datetime.datetime.strptime(f'{year}/{month}/{day} {hour}', '%Y/%m/%d %H')
        file_dates.append(date)
    return file_dates

# Function searches for the required files in parallel
def get_results_of_files_plus5(file_list):
    def fetch_file(bad_file):
        year = f"{bad_file.year:04d}"
        month = f"{bad_file.month:02d}"
        day = f"{bad_file.day:02d}"
        hour = f"{bad_file.hour:02d}"
        
        new_date = f'{year}/{month}/{day} {hour}:05:00'
        new_date_10 = f'{year}/{month}/{day} {hour}:10:00'

        new_time_range = a.Time(new_date, new_date_10)
        new_results = Fido.search(
            new_time_range,
            a.Instrument.aia,
            a.Wavelength(193 * u.angstrom),
            a.Sample(60 * u.minute)
        )
        return new_results

    list_of_new_files = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(fetch_file, file_list), total=len(file_list), desc="Fetching files"))
        for result in results:
            list_of_new_files.append(result)

    return list_of_new_files


# download the new files
def download_new_files(list_of_new_files, download_directory, redownload=False):
    def fetch_and_save(results):
        new_downloaded_files = Fido.fetch(results, path=download_directory)
        # Save the list of newly downloaded files to a text file
        if redownload:
            file_name = "redownloaded_files.txt"
        else:
            file_name = "new_downloaded_files.txt"
        new_downloaded_files_file = os.path.join(download_directory, file_name)
        with open(new_downloaded_files_file, 'a') as f:
            for file in new_downloaded_files:
                f.write(f"{file}\n")

    with ThreadPoolExecutor(max_workers=5) as executor:
        list(tqdm(executor.map(fetch_and_save, list_of_new_files), total=len(list_of_new_files), desc="Downloading files"))
    


# MAIN CODE
# List all files in the download directory
aia_files = [os.path.join(download_directory, f) for f in os.listdir(download_directory) if f.endswith('.fits')]


# Get list of missing files
missing_files = get_missing_files(aia_files, start_date, end_date)
print("Number of missing files:", len(missing_files))
# Download new files for the missing files
print("Search for missing files")
results = get_results_of_files_plus5(missing_files)
print("Downloading missing Files")
download_new_files(results, download_directory, redownload=False)

# List all files in the download directory
aia_files = [os.path.join(download_directory, f) for f in os.listdir(download_directory) if f.endswith('.fits')]

# check and delete bad quality files
bad_quality_files = check_file_quality(aia_files, download_directory)
print("Number of bad quality files:", len(bad_quality_files))
print("Searching for new files")
# download new files for the bad quality files
bad_quality_files = convert_file_list_to_datetime(bad_quality_files)
results = get_results_of_files_plus5(bad_quality_files)
print("Downloading new files")
download_new_files(results, download_directory, redownload=True)

# Open the new_downloaded_files.txt and read the contents
new_downloaded_files_file = os.path.join(download_directory, 'redownloaded_files.txt')
with open(new_downloaded_files_file, 'r') as f:
    new_downloaded_files = f.readlines()

# Remove the newline character from the file names
new_downloaded_files = [file.strip() for file in new_downloaded_files]

# Print the list of newly downloaded files
print("Newly downloaded files:")
for file in new_downloaded_files:
    print(file)

# check and delete bad quality files
bad_quality_files = check_file_quality(new_downloaded_files, download_directory)

print("New files with bad quality:")
for file in bad_quality_files:
    print(file)

Number of missing files: 48
Search for missing files


Fetching files:   0%|          | 0/48 [00:00<?, ?it/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

...

Bad quality file: C:\Sun Data\aia.lev1.193A_2013_01_02T20_05_05.46Z.image_lev1.fits
Number of bad quality files: 1
Searching for new files


Fetching files:   0%|          | 0/1 [00:00<?, ?it/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia.lev1.193A_2013_01_02T20_05_05.46Z.image_lev1.fits:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Newly downloaded files:
C:\Sun Data\aia.lev1.193A_2013_01_02T20_05_05.46Z.image_lev1.fits
Bad quality file: C:\Sun Data\aia.lev1.193A_2013_01_02T20_05_05.46Z.image_lev1.fits
New files with bad quality:
C:\Sun Data\aia.lev1.193A_2013_01_02T20_05_05.46Z.image_lev1.fits


### Preprocessing 

#### Pointing correction

#### Pointing correction can be done together with image registration 

In [7]:

# # List all files in the download directory
# aia_files = [f for f in os.listdir(download_directory) if f.endswith('.fits')]

# # update pointing table for all fits files in the download directory
# for aia_file in tqdm(aia_files, desc="Updating pointing"):
#     aia_file_path = os.path.join(download_directory, aia_file)
#     # Open the FITS file
#     try:
#         hdul = fits.open(aia_file_path, mode="update")
#         header = hdul[1].header
#         data = hdul[1].data
        
#     except Exception as e:
#         print(f"FILE CORRUPTED: {aia_file_path}, Error: {e}")
#         continue
#     # remove these two header keywords because they are nan and are causing errors with sunpy map
#     header.remove('OSCNMEAN', ignore_missing=True)
#     header.remove('OSCNRMS', ignore_missing=True)
#     # convert to sunpy map
#     aia_map = sunpy.map.Map((data, header))
#     # download pointing table
#     pointing_table = get_pointing_table("JSOC", time_range=(aia_map.date - 12 * u.h, aia_map.date + 12 * u.h))
#     # update pointing
#     aia_map_updated_pointing = update_pointing(aia_map, pointing_table=pointing_table)
#     hdul[1].header.update(aia_map_updated_pointing.meta)
#     hdul.close()
    
    



#### Define the list of all fits files in the directory

In [4]:
# List all files in the download directory
aia_files = [f for f in os.listdir(download_directory) if f.endswith('.fits')]

#### PSF Deconvolution

In [ ]:

def deconvolve_file(aia_file, psf=None):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"

    header.remove('OSCNMEAN', ignore_missing=True)
    header.remove('OSCNRMS', ignore_missing=True)

    aia_map = sunpy.map.Map((data, header))
    if (psf is None):
        psf = aiapy.psf.psf(aia_map.wavelength)
    aia_map_deconvolved = aiapy.psf.deconvolve(aia_map, psf=psf)

    hdul[1].data = aia_map_deconvolved.data
    hdul.close()
    return psf

psf = deconvolve_file(aia_files[0])
aia_files = aia_files[1:]

with ThreadPoolExecutor(max_workers=2) as executor:
    list(tqdm(executor.map(lambda file: deconvolve_file(file, psf=psf), aia_files), total=len(aia_files), desc="PSF Deconvolution"))


### Pointing Correction and Image Registration 

In [ ]:
def process_file(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"
    header.remove('OSCNMEAN', ignore_missing=True)
    header.remove('OSCNRMS', ignore_missing=True)
    aia_map = sunpy.map.Map((data, header))
    pointing_table = get_pointing_table("JSOC", time_range=(aia_map.date - 12 * u.h, aia_map.date + 12 * u.h))
    aia_map_updated_pointing = update_pointing(aia_map, pointing_table=pointing_table)
    aia_map_registered = register(aia_map_updated_pointing)

    hdul[1].data = aia_map_registered.data
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(process_file, aia_files), total=len(aia_files), desc="Pointing Correction and Image Registration"))


### Degradation Correction

In [ ]:
correction_table = get_correction_table("JSOC")
print("Got correction table")

def correct_degradation_file(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"

    aia_map = sunpy.map.Map((data, header))
    aia_map_corrected = correct_degradation(aia_map, correction_table=correction_table)
    hdul[1].data = aia_map_corrected.data
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(correct_degradation_file, aia_files), total=len(aia_files), desc="Degradation Correction"))


### Exposure Normalization

In [ ]:
def normalize_by_exposure(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"
    
    aia_map = sunpy.map.Map((data, header))
    aia_map_normalized = aia_map / aia_map.exposure_time
    hdul[1].data = aia_map_normalized.data
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(normalize_by_exposure, aia_files), total=len(aia_files), desc="Normalizing by Exposure Time"))


#### Clipping values between 100 and 5000

In [ ]:
def clip_values(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"
    
    # Clip the values between 100 and 5000
    data_clipped = data.clip(100, 5000)
    
    # Save the clipped image
    hdul[1].data = data_clipped
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(clip_values, aia_files), total=len(aia_files), desc="Clipping Values"))

#### Rescaling Images by Log10 (Cant do this because it causes issues while copying)

In [ ]:
# for aia_file in tqdm(aia_files, desc="Rescaling to log10"):
#     aia_file_path = os.path.join(download_directory, aia_file)
#     # Open the FITS file
#     try:
#         hdul = fits.open(aia_file_path, mode="update")
#         header = hdul[1].header
#         data = hdul[1].data
#     except Exception as e:
#         print(f"FILE CORRUPTED: {aia_file_path}, Error: {e}")
#         continue
    
#     # Rescale the data to log10 scale
#     data_log10 = np.log10(data + 1)  # Adding 1 to avoid log(0)
    
#     # Save the rescaled image
#     hdul[1].data = data_log10
    
#     hdul.close()

#### Resizing images to 224 x 224 using pillow/opencv with bilinear interpolation

In [ ]:

for aia_file in tqdm(aia_files, desc="Downscaling to 224x224"):
    aia_file_path = os.path.join(download_directory, aia_file)
    # Open the FITS file
    # Specify the new directory where you want to copy the files
    new_directory = os.path.join(download_directory, 'non_anti_aliasing/')
    os.makedirs(new_directory, exist_ok=True)

    # Copy the file to the new directory
    new_file_path = os.path.join(new_directory, aia_file)
    shutil.copy2(aia_file_path, new_file_path)

    # open first file
    try:
        hdul1 = fits.open(aia_file_path, mode="update")
        header1 = hdul1[1].header
        data1 = hdul1[1].data
    except Exception as e:
        print(f"FILE CORRUPTED: {aia_file_path}, Error: {e}")
        continue
    
    # Rescale the data to log10 scale AGAIN BECUASE FOR SOME REASON COPYING THE FILE UNDOS IT!!!
    data_log10_1 = np.log10(data1 + 1)  # Adding 1 to avoid log(0)
    # Using pillow or anti-aliasing
    image_pil = Image.fromarray(data_log10_1)
    resized_data_aa = image_pil.resize((224, 224), Image.BILINEAR)
    resized_image_np = np.array(resized_data_aa)
    # Save the downscaled anti-aliased image
    hdul1[1].data = resized_image_np
    
    hdul1.close()
    
    # Open second file
    try:
        hdul2 = fits.open(new_file_path, mode="update")
        header2 = hdul2[1].header
        data2 = hdul2[1].data
    except Exception as e:
        print(f"FILE CORRUPTED: {new_file_path}, Error: {e}")
        continue
    
    # Rescale the data to log10 scale AGAIN BECUASE FOR SOME REASON COPYING THE FILE UNDOS IT!!!
    data_log10_2 = np.log10(data2 + 1)  # Adding 1 to avoid log(0)
    
    # Using OpenCV for non-anti-aliasing
    resized_data = cv2.resize(data_log10_2, (224, 224), interpolation=cv2.INTER_LINEAR)
    # Save the downscaled non-anti-aliased image
    hdul2[1].data = resized_data


    hdul2.close()
    
